# One step further: Mastering Atari games with RL

## Introduction

## Initialization

In [ ]:
import gym
import torch
import matplotlib.pyplot as plt

from gym.wrappers import AtariPreprocessing, FrameStack
from core.nn.utils import TensorBoardEnv, PytorchEnv, NormalizeObservationEnv

## Creating the environment

In [ ]:
def make_atari(env_name: str, tag: str) -> gym.Env:
    env = gym.make(env_name)
    env = AtariPreprocessing(env)
    env = FrameStack(env, num_stack=4)
    env = TensorBoardEnv(env, tag=tag)
    env = PytorchEnv(env)
    env = NormalizeObservationEnv(env)
    return env

In [ ]:
env = make_atari('BreakoutNoFrameskip-v4', tag='')

## What actions can our agent take?

In [ ]:
meanings = env.unwrapped.get_action_meanings()
actions = list(zip(range(len(meanings)), meanings))
print(f"These are the actions we can take and what they mean:\n\n{actions}")

## What a human sees:

In [ ]:
# Take several random actions to fetch a random state of the game.

state = env.reset()
for _ in range(100):
    random_action = torch.randint(env.action_space.n, (1, 1))
    state, _, _, _ = env.step(random_action)

frame = env.render(mode='rgb_array')

In [ ]:
plt.figure(figsize=(8,8))
plt.title("Human observation of the game")
plt.imshow(frame)

## What our agent sees:

In [ ]:
# Take several random actions to fetch a random state of the game.

state = env.reset()
for _ in range(10):
    random_action = torch.randint(env.action_space.n, (1, 1))
    state, _, _, _ = env.step(random_action)

In [ ]:
# Check the shape of an observation.
print(
    "Each observation has the following shape:", 
    "Batch x Frames x Height x Width",
    state.shape,
    sep="\n")

In [ ]:
# Separate the frames.
frame1, frame2, frame3, frame4 = state.squeeze()

In [ ]:
# Put the frames side by side.
merged_frames = torch.cat([frame1, frame2, frame3, frame4], dim=1)

In [ ]:
plt.figure(figsize=(16,16))
plt.title("Sample Observation")
plt.imshow(merged_frames, cmap='gray')

## Conclusions

## References